In [1]:
import urllib.request
import urllib.parse
import requests
import re
from collections import Counter

## 用``urllib.request``库发起``get``请求，输出``response``的文本信息

In [2]:
url ='http://httpbin.org'
response  = urllib.request.urlopen(url)
# print(response.status) #响应头状态码
# print(response.getheaders()) # 响应头信息
# print(response.getheader('Server')) 
print(response.read().decode('utf-8'))

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <title>httpbin.org</title>
    <link href="https://fonts.googleapis.com/css?family=Open+Sans:400,700|Source+Code+Pro:300,600|Titillium+Web:400,600,700"
        rel="stylesheet">
    <link rel="stylesheet" type="text/css" href="/flasgger_static/swagger-ui.css">
    <link rel="icon" type="image/png" href="/static/favicon.ico" sizes="64x64 32x32 16x16" />
    <style>
        html {
            box-sizing: border-box;
            overflow: -moz-scrollbars-vertical;
            overflow-y: scroll;
        }

        *,
        *:before,
        *:after {
            box-sizing: inherit;
        }

        body {
            margin: 0;
            background: #fafafa;
        }
    </style>
</head>

<body>
    <a href="https://github.com/requests/httpbin" class="github-corner" aria-label="View source on Github">
        <svg width="80" height="80" viewBox="0 0 250 250" style="fill:#151513; color:#fff; position: absolute;

## 用``requests``库伪装成火狐浏览器发起``post``请求,输出``response``文本信息

In [3]:
url = 'http://httpbin.org/post'
headers = {
    'User-Agent':'Mozilla/4.0 (compatible; MSIE S. S; Windows NT)',# 伪装成火狐浏览器
    'Host':'httpbin.org'
}
# headers 指定user-Agent,host
req = requests.post(url =url,headers=headers)
print(req.text) # 输出文本信息

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "0", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/4.0 (compatible; MSIE S. S; Windows NT)", 
    "X-Amzn-Trace-Id": "Root=1-5e636dfb-f37b3d027e1d7d98334a32f0"
  }, 
  "json": null, 
  "origin": "113.220.6.35", 
  "url": "http://httpbin.org/post"
}



## 用``request``库中的``Session``对象发出``get``请求，设置cookies并获取，输出获取的cookies

In [4]:
url = 'http://httpbin.org/cookies/set/times/2014-02-12'
s = requests.Session() # 维持同一个会话
s.get(url) #设置该会话一个cookies
reponse = s.get('http://httpbin.org/cookies') # 获取当前cookies
print(reponse.text)

{
  "cookies": {
    "times": "2014-02-12"
  }
}



## 实现爬取网页的不去重深度算法

In [5]:
count = 3 #爬取网页个数
r = re.compile(r'href=[\`"]?(http[^\`">]+)') # 匹配网页的正则表达式
seed = 'http://httpbin.org/'       # 初始网页
queue =[seed]                      # 待爬取网页栈
used = set()                       # 已爬取网页集合
storge = {}                        # 存储网页对应内容
while len(queue)>0 and count>0:
    try:                           #异常处理
        url = queue.pop(-1)        # 从末尾删除元素
        html = requests.get(url).text
        storge[url] = html
        new_urls = r.findall(html) # 找出上一个爬取网站内容中所含的所有网址信息
        used.add(url)              # 将已爬取的网址加入到used中
        for new_url in new_urls:
            queue.append(new_url)  # 从末尾添加，将所有网址添加到待爬取队列，先进后出形成栈
        count-=1
    except Exception as e:        # 出现异常时输出异常的网站，及异常类型
        print(url)
        print(e)

In [6]:
number =Counter(queue)
for url,count in number.most_common():
    if count>1:
        print(url,count)
# 输出为待爬取网址队列中重复出现的网址，及重复次数

https://education.github.com 4
https://github.githubassets.com/assets/site-db5797d9868460861af755858749976b.css 3
https://lab.github.com/ 3
https://github.community 3
https://desktop.github.com/ 3
https://github.blog 3
https://github.githubassets.com 2
https://avatars0.githubusercontent.com 2
https://avatars1.githubusercontent.com 2
https://avatars2.githubusercontent.com 2
https://avatars3.githubusercontent.com 2
https://github-cloud.s3.amazonaws.com 2
https://user-images.githubusercontent.com/ 2
https://github.githubassets.com/assets/frameworks-7b00555b34e27547e4200fc646d0f990.css 2
https://github.githubassets.com/assets/github-cf76c0d32d2339066765a3b53a4a84d4.css 2
https://github.com/fluidicon.png 2
https://github.githubassets.com/ 2
https://github.githubassets.com/pinned-octocat.svg 2
https://github.githubassets.com/favicon.ico 2
https://github.com/ 2
https://opensource.guide 2
https://github.com/events 2
https://enterprise.github.com/contact 2
http://flasgger.pythonanywhere.com/ 2


运行代码后，``queue``存储了尚未爬取的网址，存在相同网站，``used``存储了已爬取网址,``storge``用字典存储爬取网站的文本信息，``key``为网站地址,``value``为对应网站文本信息

##  抓取网页的广度遍历算法

## 去重

In [7]:
count = 3                                    # 爬取网页数
r = re.compile(r'href=[\`"]?(http[^\`">]+)') # 匹配网页正则表达式
seed = 'http://httpbin.org/'                 # 初始网页
queue =[seed]                                # 待爬取队列
used = set()                                 #  已爬取网址集合
storge = {}                                  #  存储已爬取网址文本内容
recount =0                                   # 重复网址出现次数
while len(queue)>0 and count>0:
    try:
        url = queue.pop(0)                   # 从头部取出
        html = requests.get(url).text
        storge[url] = html                   # 存储网址文本内容
        new_urls = r.findall(html)
        used.add(url)
        
        for new_url in new_urls:
            if new_url not in used and new_url not in queue:
                queue.append(new_url)        # 从尾部添加数据，形成队列
            else:
                print('重复： '+url)         # 输出重复出现的网址
                recount+=1                   # 重复网址数加一
        count-=1
    except Exception as e:                  # 异常处理，输出出现异常的网址和错误
        print(url)
        print(e)

重复： https://github.com/requests/httpbin
重复： https://github.com/requests/httpbin
重复： https://github.com/requests/httpbin
重复： https://github.com/requests/httpbin


运行代码后，会自行输出爬取时出现的重复网站，``queue``存储了尚未爬取的网址，``used``存储了已爬取网址,``storge``用字典存储爬取网站的文本信息，``key``为网站地址,``value``为对应网站文本信息

## 不去重广度算法

In [8]:
count = 3                                    # 爬取网页数
r = re.compile(r'href=[\`"]?(http[^\`">]+)') # 匹配网页正则表达式
seed = 'http://httpbin.org/'                 # 初始网页
queue =[seed]                                # 待爬取队列
used = set()                                 #  已爬取网址集合
storge = {}                                  #  存储已爬取网址文本内容
while len(queue)>0 and count>0:
    try:
        url = queue.pop(0)                   # 从头部取出
        html = requests.get(url).text
        storge[url] = html                   # 存储网址文本内容
        new_urls = r.findall(html)
        used.add(url)
        for new_url in new_urls:
            queue.append(new_url)        # 从尾部添加数据，形成队列
        count-=1
    except Exception as e:                  # 异常处理，输出出现异常的网址和错误
        print(url)
        print(e)

In [9]:
number =Counter(queue)
for url,count in number.most_common():
    if count>1:
        print(url,count)
# 输出为待爬取网址队列中重复出现的网址，及其重复次数

https://education.github.com 2
https://httpbin.org 2
https://desktop.github.com/ 2
http://httpbin.org 2


运行代码后，``queue``存储了尚未爬取的网址，``used``存储了已爬取网址,``storge``用字典存储爬取网站的文本信息，``key``为网站地址,``value``为对应网站文本信息

## 将网页的去重深度遍历算法封装成类，并测试

In [8]:
import requests,re
from collections import Counter
class url_get():
    def __init__(self,url):
        
        self.url = url   # url为初始网址
        self.storge = {} # 存储网址对应的text
        self.recount = 0 # 记录重复网址数
        self.queue = [self.url]
        self.used = []
        
    def get_url(self,temp):
        self.html = requests.get(temp).text
        # 该网站内的信息 
        self.storge[temp] = self.html
        # 存储数据 字典
        r = re.compile(r'href=[\`"]?(http[^\`">]+)')
        self.new_urls = r.findall(self.html)
        # 该网址信息中所含的其他网址
    def DFS(self,count): 
        # 深度探索
        self.count =count # count为爬取网址个数
        while len(self.queue)>0 and self.count >0:
            tem = self.queue.pop(-1)
            self.get_url(tem)
            self.used.append(tem)
            for new_url in self.new_urls:
                if new_url not in self.used and new_url not in self.queue:
                    self.queue.append(new_url)
                
                else:
                    self.recount+=1
            self.count-=1
    def count_queue(self):
        url_count = Counter(self.queue)
        for url,count in url_count.most_common(10):
            print(url,count)

In [9]:
url = 'http://httpbin.org/'
data=url_get(url) # 输入网址
data.DFS(3) # 爬取网址个数

In [12]:
data.recount # 重复网站出现次数

37

In [13]:
data.used # 爬取的网址

['http://httpbin.org/',
 'https://github.com/rochacbruno/flasgger',
 'https://github.com/about']